In [7]:
import pandas as pd

# Let's load the .csv into a dataframe
PGdata = pd.read_csv(r'C:\Users\Administrator\Data Analytics\Applied Statistics\PlantGrowth.csv') 

# Display the DataFrame information - the info() method provides information about the dataset, including the number of rows, columns, column names, data types, and memory usage.

PGdata.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   rownames  30 non-null     int64  
 1   weight    30 non-null     float64
 2   group     30 non-null     object 
dtypes: float64(1), int64(1), object(1)
memory usage: 852.0+ bytes
